### 为每行数据找到相似度最高的新闻
结果存在 new 后缀的表中

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
image_path = '/data1/cjx/fakeddit_images'
data_path = './fakeddit'
save_path = './fakeddit'
train_path = f"{data_path}/train.tsv"
valid_path = f"{data_path}/valid.tsv"
test_path = f"{data_path}/test.tsv"
train_df = pd.read_csv(train_path, sep='\t')
valid_df = pd.read_csv(valid_path, sep='\t')
test_df = pd.read_csv(test_path, sep='\t')

In [2]:
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(28180, 16)
(2965, 16)
(2964, 16)


In [3]:
spload = spacy.load('en_core_web_trf')

In [20]:
import warnings
warnings.filterwarnings("ignore")

def new_data(data_frame: pd.DataFrame):
    df1 = data_frame[data_frame['2_way_label'] == 1].reset_index(drop=True)
    df0 = data_frame[data_frame['2_way_label'] == 0].reset_index(drop=True)

    df1_spacy = [spload(data['title']) for i, data in df1.iterrows()]
    df0_spacy = [spload(data['title']) for i, data in df0.iterrows()]

    # 各取第五个
    k = 5
    k1 = df1['id'][k]
    k0 = df0['id'][k]
    start_df = data_frame.loc[data_frame.isin([k1, k0]).any(axis=1)]
    # 取最后一个 index
    s = start_df.index[-1]
    
    reff = pd.DataFrame()
    for i in tqdm(range(s, len(data_frame))):
        # 遍历 df
        title_I = data_frame['title'][i]
        doc = spload(title_I)

        # 取出在本条数据之前的所有数据
        ref1 = df1[(df1.index < i)]
        ref0 = df0[(df0.index < i)]
        # 将之前的所有数据都与本数据的 title，计算相似度
        ref1_sim = [doc.similarity(df1_spacy[j]) for j in range(len(ref1))]
        ref0_sim = [doc.similarity(df0_spacy[j]) for j in range(len(ref0))]

        ref1['sim'] = ref1_sim
        ref0['sim'] = ref0_sim

        # 按 title 相似度进行降序排序，取前五个
        ref1_id = ref1.sort_values(by=['sim'], ascending=False).head(k)['id']
        # 将拼接完后的数据转为一行，并加上列名
        ref1_list = ref1_id.to_frame().T
        ref1_list.columns = ['i11', 'i12', 'i13', 'i14', 'i15']

        ref0_id = ref0.sort_values(by=['sim'], ascending=False).head(k)['id']
        ref0_list = ref0_id.to_frame().T
        ref0_list.columns = ['i01', 'i02', 'i03', 'i04', 'i05']

        # 至此，每条数据都在真假新闻中找到了最相似的各 5 条
        # 将这 10 条的信息全都存在同一行里，通过 id 来索引出这些数据
        ref = pd.concat([ref1_list, ref0_list], axis=1).reset_index(drop=True)
        ref['Id'] = data_frame['id'][i]
        reff = pd.concat([reff, ref], axis=0)
    
    reff.reset_index(drop=True)
    return reff

In [25]:
# 先取子集测试一下
sample_test = test_df.sample(frac=0.1).reset_index(drop=True)
sample_test_new_df = new_data(sample_test)
sample_test_new_df

100%|██████████| 282/282 [00:10<00:00, 26.03it/s]


In [27]:
test_new_df = new_data(test_df)
test_new_df.to_csv(f"{data_path}/test_new.tsv", sep='\t', index=False)
test_new_df

100%|██████████| 2953/2953 [03:06<00:00, 15.84it/s]


In [30]:
valid_new_df = new_data(valid_df)
valid_new_df.to_csv(f"{data_path}/valid_new.tsv", sep='\t', index=False)
valid_new_df

100%|██████████| 2949/2949 [03:06<00:00, 15.81it/s]


,i11,i12,i13,i14,i15,i01,i02,i03,i04,i05,Id
0,d1zybe,bg0vzf,8gq24i,c7sq2z,64v0a6,c6fp7pw,djjw878,ckhqliy,brjk5i,2410qm,d1zybe
0,bg0vzf,bvey9g,37ld7z,74qnsg,cm1d43,90j5n3,c6fp7pw,d4mkm72,e4k481p,9diunr,90j5n3
0,bg3q34,bg0vzf,2249xx,cutrnx,37ld7z,c6fp7pw,djjw878,6uj9jf,e4k481p,9diunr,bg3q34
0,c61tn8,bg0vzf,2249xx,80a0gy,cutrnx,c6fp7pw,58597u,com1b8,6uj9jf,e4k481p,c61tn8
0,cbkt52,bg0vzf,9hkvc2,1gmzqo,80a0gy,c6fp7pw,djjw878,cnhvvaq,com1b8,6uj9jf,cbkt52
...,...,...,...,...,...,...,...,...,...,...,...
0,4a0ygz,bg0vzf,4v4weh,34bf5s,bnii28,c6fp7pw,8pz7mp,cnijpmt,d7jgkbj,c7xzcm8,4a0ygz
0,bg0vzf,b32v4z,34bf5s,bnii28,cpdb96,3a7685,c6fp7pw,8pz7mp,cnijpmt,d7jgkbj,3a7685
0,ctje18,b32v4z,34bf5s,bnii28,cpdb96,c6fp7pw,8pz7mp,cnijpmt,d7jgkbj,c7xzcm8,ctje18
0,bg0vzf,b32v4z,34bf5s,bnii28,cpdb96,c6trw3q,c6fp7pw,8pz7mp,cnijpmt,d7jgkbj,c6trw3q


In [31]:
train_new_df = new_data(train_df)
train_new_df.to_csv(f"{data_path}/train_new.tsv", sep='\t', index=False)
train_new_df

100%|██████████| 28164/28164 [2:02:20<00:00,  3.84it/s]  


,i11,i12,i13,i14,i15,i01,i02,i03,i04,i05,Id
0,3mec8i,73u8p7,aw55am,b0wuix,2my28w,c49r5ki,8bgzk5,cvt0if9,911dhn,b15ysv,3mec8i
0,73u8p7,4axhlf,c5zkjs,co6coo,1zv1pp,76tfzo,c49r5ki,9ejhpn,cj52438,cvvxm2e,76tfzo
0,73u8p7,aw55am,5n01a0,c5zkjs,co6coo,9gl4pk,c49r5ki,8bgzk5,c7yqrcx,cj52438,9gl4pk
0,b0lqoi,73u8p7,27kbqf,dgj3p2,5n01a0,c49r5ki,crul7za,c7dmel3,c7yqrcx,cj52438,b0lqoi
0,cy96p6,73u8p7,c3c7ek,12j182,dgj3p2,c49r5ki,8bgzk5,5c0yym,c7dmel3,c7yqrcx,cy96p6
...,...,...,...,...,...,...,...,...,...,...,...
0,c0jgjv,73u8p7,2urfsl,b5jnfe,5j9jdw,c49r5ki,cfjq5ow,d2vktma,589wih,6kiux3,c0jgjv
0,73u8p7,2urfsl,b5jnfe,5j9jdw,9gvtdu,7kchrw,c49r5ki,clury9f,cajzr6t,cjyyzts,7kchrw
0,arzzrg,73u8p7,2urfsl,b5jnfe,5j9jdw,c49r5ki,cfjq5ow,d2vktma,589wih,6kiux3,arzzrg
0,73u8p7,2urfsl,b5jnfe,5j9jdw,9gvtdu,axqv3u,cfjq5ow,d2vktma,589wih,6kiux3,axqv3u


### 为img进行切片，CNN会用到

In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models.resnet import ResNet50_Weights
image_path = '/data1/cjx/fakeddit_images'
data_path = './fakeddit'
save_path = './fakeddit/images'
train_path = f"{data_path}/train.tsv"
valid_path = f"{data_path}/valid.tsv"
test_path = f"{data_path}/test.tsv"
train_df = pd.read_csv(train_path, sep='\t')
valid_df = pd.read_csv(valid_path, sep='\t')
test_df = pd.read_csv(test_path, sep='\t')

In [6]:
all_df = pd.concat([train_df, valid_df, test_df]).reset_index(drop=True)
all_df.to_csv(f"{data_path}/all_data.tsv", sep='\t', index=False)

In [20]:
arr = all_df.iloc[np.array([1, 3, 4])].values
arr[[0, 2], 1]

array(['a man in a dinosaur costume throwing the first pitch at a baseball game',
       'i have no position on woody allen'], dtype=object)

In [2]:
# all images
id_arr = pd.concat([train_df["id"], valid_df["id"], test_df["id"]]).values
id_arr.shape

(34109,)

In [3]:
image_ids = list(map(lambda image_id: f"{image_path}/{image_id}.jpg", id_arr))
len(image_ids)

34109

In [4]:
resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet50 = nn.Sequential(*list(resnet.children())[:-1])

for param in resnet50.parameters():
    param.requires_grad = False

In [5]:
def cut_image(image: Image):
    width, height = image.size
    item_width = int(width / 14)
    box_list = []
    # 与 vit 相同的 patch 数
    for i in range(0, 14):
        for j in range(0, 14):
            box = (j * item_width, i * item_width, (j + 1) * item_width, (i + 1) * item_width)
            box_list.append(box)
    # box 只是 pixel index，得从img上切下来
    patch_list = [np.array(image.crop(box)) for box in box_list]
    return np.array(patch_list)

In [18]:
# 用 0 图片测试一下
img = Image.open(image_ids[0]).convert('RGB')
img = img.resize((224, 224), Image.Resampling.LANCZOS)
patch_list = cut_image(img)
patches_tensor = torch.tensor(patch_list)
patches_tensor = patches_tensor.permute(0, 3, 1, 2).float()
patches_feature = resnet50(patches_tensor)
# 压缩 patch
patches_feature = patches_feature.squeeze(3).squeeze(2).numpy()

In [19]:
patch0_img = Image.fromarray(patch_list[0].astype('uint8')).convert('RGB')
patch0_img

In [6]:
# def preprocess_all_images():
all_patches_feature = []
for image_id_path in tqdm(image_ids):
    img = Image.open(image_id_path).convert('RGB')
    img = img.resize((224, 224), Image.Resampling.LANCZOS)
    patch_list = cut_image(img)
    # (196, 16, 16, 3)
    patches_tensor = torch.tensor(patch_list)
    # (196, 3, 16, 16)
    patches_tensor = patches_tensor.permute(0, 3, 1, 2).float()
    patches_feature = resnet50(patches_tensor)
    # (196, 2048)
    patches_feature = patches_feature.squeeze(3).squeeze(2).numpy()
    all_patches_feature.append(patches_feature)
# 直接 ndarray
images_196p = pd.DataFrame({'Id': id_arr.tolist(), 'img': all_patches_feature})

  2%|▏         | 543/34109 [01:03<1:05:40,  8.52it/s]/data1/cjx/anaconda3/envs/multimodal-news/lib/python3.9/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 34109/34109 [1:12:42<00:00,  7.82it/s]


In [13]:
np.save(f'{save_path}/image_196_for_r50.npy', images_196p)

In [10]:
img0_feature = images_196p["img"].iloc[0]
img0_feature.shape
# (34109, (1, (196, 2048)))

(196, 2048)

In [3]:
# 读出来成 df
img_features = np.load(f'{save_path}/image_196_for_r50.npy', allow_pickle=True)

In [9]:
img_features[:, 0]

array(['c49r5ki', '73u8p7', '8bgzk5', ..., '5vuug0', 'ay8wtp', 'd3ie3qu'],
      dtype=object)

In [15]:
result_arr = np.char.find(img_features[:, 0].astype(str), "d3ie3qu")
result_arr

array([-1, -1, -1, ..., -1, -1,  0])

In [27]:
feature_idx = int(np.argwhere(result_arr == 0))
feature_idx

34108

In [29]:
img_features[feature_idx, 1].shape

(196, 2048)